<a href="https://colab.research.google.com/github/Gabrielle-Robinson/Covid-19-Cases-In-India/blob/main/Covid_19_in_India.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First download data from https://www.kaggle.com/anandhuh/latest-covid19-india-statewise-data and upload india_covid_data.zip
!unzip india-covid-data.zip

In [ ]:
# Installing PySpark:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# Installing PySpark:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
# Installing PySpark:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
# Load data as a dataframe in pyspark 
df = spark.read.csv('/content/Latest Covid-19 India Status.csv', inferSchema=True, header=True)
df

State/UTs,Total Cases,Active,Discharged,Deaths,Active Ratio,Discharge Ratio,Death Ratio,Population
Andaman And Nicobar,10017,10,9878,129,0.1,98.61,1.29,100896618
Andhra Pradesh,2317741,2850,2300165,14726,0.12,99.24,0.64,128500364
Arunachal Pradesh,64413,149,63968,296,0.23,99.31,0.46,658019
Assam,724110,1569,715903,6638,0.22,98.87,0.92,290492
Bihar,830062,321,817486,12255,0.04,98.48,1.48,40100376
Chandigarh,91732,165,90402,1165,0.18,98.55,1.27,79502477
Chhattisgarh,1150807,1705,1135075,14027,0.15,98.63,1.22,28900667
Dadra And Nagar H...,11438,1,11433,4,0.01,99.96,0.03,231502578
Delhi,1859634,2086,1831426,26122,0.11,98.48,1.4,773997
Goa,245001,301,240899,3801,0.12,98.33,1.55,3772103


In [ ]:
# Replace spaces from column names nd replace with underscore to increase pyspark compatiblity
# Name this new dataframe "renamed"
from pyspark.sql import functions as F 

renamed_df = df.select([F.col(col).alias(col.replace(' ', "_")) for col in df.columns])
renamed_df

# Replace "/" symbol with underscore
renamed_df = renamed_df.withColumnRenamed('State/UTs', 'State_UTs')

In [ ]:
# Convert renamed dataframe to use SQL syntax and name it 'Data'
renamed_df.createOrReplaceTempView('Data')

In [ ]:
spark.sql('SELECT * FROM Data')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
Andaman And Nicobar,10017,10,9878,129,0.1,98.61,1.29,100896618
Andhra Pradesh,2317741,2850,2300165,14726,0.12,99.24,0.64,128500364
Arunachal Pradesh,64413,149,63968,296,0.23,99.31,0.46,658019
Assam,724110,1569,715903,6638,0.22,98.87,0.92,290492
Bihar,830062,321,817486,12255,0.04,98.48,1.48,40100376
Chandigarh,91732,165,90402,1165,0.18,98.55,1.27,79502477
Chhattisgarh,1150807,1705,1135075,14027,0.15,98.63,1.22,28900667
Dadra And Nagar H...,11438,1,11433,4,0.01,99.96,0.03,231502578
Delhi,1859634,2086,1831426,26122,0.11,98.48,1.4,773997
Goa,245001,301,240899,3801,0.12,98.33,1.55,3772103


In [ ]:
# Calculate how many State/Union Territories there are
spark.sql('SELECT COUNT(State_UTs) FROM Data')

count(State_UTs)
36


In [ ]:
# Calculate which State/Union Territories have the highest number of active cases
spark.sql('SELECT * FROM Data ORDER BY Active desc LIMIT 5')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
Kerala,6497204,30745,6401236,65223,0.47,98.52,1.0,91702478
Maharashtra,7865298,11225,7710376,143697,0.14,98.03,1.83,399001
Mizoram,213328,7268,205403,657,3.41,96.29,0.31,1308967
Karnataka,3940795,6526,3894333,39936,0.17,98.82,1.01,1711947
Tamil Nadu,3449007,6393,3404611,38003,0.19,98.71,1.1,35998752


In [ ]:
# Categorize where the highest death ratio is to decipher where COVID is the most detrimental
spark.sql('SELECT * FROM Data ORDER BY Death_Ratio desc LIMIT 5')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
Punjab,758047,729,739616,17702,0.1,97.57,2.34,34698876
Nagaland,35402,119,34528,755,0.34,97.53,2.13,38157311
Maharashtra,7865298,11225,7710376,143697,0.14,98.03,1.83,399001
Uttarakhand,436074,2392,426002,7680,0.55,97.69,1.76,85002417
Meghalaya,93473,196,91699,1578,0.21,98.1,1.69,30501026


In [ ]:
# Calculate how many people in total got infected and how many people in total passed away 
spark.sql('SELECT SUM(Total_Cases), SUM(Deaths) FROM Data')

sum(Total_Cases),sum(Deaths)
42924130,513843


In [ ]:
# Calculate which State/Union Territories have the lowest number of active cases/ safest places to be
spark.sql('SELECT * FROM Data ORDER BY Active asc LIMIT 5')

State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
Dadra And Nagar H...,11438,1,11433,4,0.01,99.96,0.03,231502578
Lakshadweep,11399,8,11339,52,0.07,99.47,0.46,11700099
Andaman And Nicobar,10017,10,9878,129,0.1,98.61,1.29,100896618
Tripura,100860,27,99914,919,0.03,99.06,0.91,1646050
Sikkim,39064,59,38562,443,0.15,98.71,1.13,83697770


In [ ]:
# Convert renamed_dataframe as a pandas dataframe to do analysis in pandas
import pandas as pd
pd_df = renamed_df.toPandas()
pd_df.head()

,State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
0,Andaman And Nicobar,10017,10,9878,129,0.10,98.61,1.29,100896618
1,Andhra Pradesh,2317741,2850,2300165,14726,0.12,99.24,0.64,128500364
2,Arunachal Pradesh,64413,149,63968,296,0.23,99.31,0.46,658019
3,Assam,724110,1569,715903,6638,0.22,98.87,0.92,290492
4,Bihar,830062,321,817486,12255,0.04,98.48,1.48,40100376


In [ ]:
# Run general statistics
pd_df.describe()

,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
count,3.600000e+01,36.000000,3.600000e+01,36.000000,36.000000,36.000000,36.000000,3.600000e+01
mean,1.192337e+06,2850.027778,1.175214e+06,14273.416667,0.298889,98.580000,1.120556,3.971861e+07
std,1.763902e+06,5415.612117,1.734995e+06,26132.181559,0.551962,0.634129,0.489629,5.050913e+07
min,1.001700e+04,1.000000,9.878000e+03,4.000000,0.010000,96.290000,0.030000,6.600100e+04
25%,9.901325e+04,165.750000,9.786025e+04,1103.500000,0.110000,98.442500,0.870000,1.695473e+06
50%,5.885325e+05,1357.000000,5.816610e+05,5976.500000,0.175000,98.670000,1.090000,2.410088e+07
75%,1.280867e+06,2946.750000,1.267097e+06,14201.750000,0.272500,98.870000,1.412500,6.979986e+07
max,7.865298e+06,30745.000000,7.710376e+06,143697.000000,3.410000,99.960000,2.340000,2.315026e+08


In [ ]:
# Run correlation matrix to decipher the linear correlation (relationship) between variables
pd_df.corr()

# Notice that discharge ratio and active ratio have a negative correlation (as one increases, the other decreases)
# Notice the the total cases and deaths has a postive correlation (as more people have COVID, the more people pass away from it) -> death has.9 which is a strong correlation


,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharge_Ratio,Death_Ratio,Population
Total_Cases,1.000000,0.790911,0.999991,0.942907,-0.075794,-0.018922,0.108159,-0.066065
Active,0.790911,1.000000,0.791400,0.635165,0.219949,-0.143354,-0.064220,0.033958
Discharged,0.999991,0.791400,1.000000,0.941572,-0.076410,-0.017076,0.106456,-0.065432
Deaths,0.942907,0.635165,0.941572,1.000000,-0.088562,-0.113846,0.246018,-0.122136
Active_Ratio,-0.075794,0.219949,-0.076410,-0.088562,1.000000,-0.666845,-0.260692,-0.160832
Discharge_Ratio,-0.018922,-0.143354,-0.017076,-0.113846,-0.666845,1.000000,-0.545546,0.292856
Death_Ratio,0.108159,-0.064220,0.106456,0.246018,-0.260692,-0.545546,1.000000,-0.198759
Population,-0.066065,0.033958,-0.065432,-0.122136,-0.160832,0.292856,-0.198759,1.000000


In [ ]:
# Import plotly to create graphs and make four Histograms for 'Total Cases', 'Deaths', 'Active', 'Discharged'
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2, subplot_titles=['Total Cases', 'Deaths', 'Active', 'Discharged'])

total_cases = go.Histogram(x=pd_df['Total_Cases'], nbinsx=20, name='Total Cases')
deaths = go.Histogram(x=pd_df['Deaths'], nbinsx=20, name='Deaths')
active = go.Histogram(x=pd_df['Active'], nbinsx=20, name='Active')
discharged = go.Histogram(x=pd_df['Discharged'], nbinsx=20, name='Discharged')

# Add histrogram to figure
fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig.show()


In [ ]:
# Notice outliers for Total Cases, Deaths, and Active
# Create boxplot to futher investigate outliers in each State Union Territories 
fig = make_subplots(rows=2, cols=2, subplot_titles=['Total Cases', 'Deaths', 'Active', 'Discharged'])

total_cases = go.Box(x=pd_df['Total_Cases'], name='Total Cases', text=pd_df['State_UTs'])
deaths = go.Box(x=pd_df['Deaths'], name='Deaths', text=pd_df['State_UTs'])
active = go.Box(x=pd_df['Active'], name='Active', text=pd_df['State_UTs'])
discharged = go.Box(x=pd_df['Discharged'], name='Discharged', text=pd_df['State_UTs'])

# Add histrogram to figure
fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig.show()



In [ ]:
# Ceate barchat to see each value for each State Union Territory
fig = make_subplots(rows=2, cols=2, subplot_titles=['Total Cases', 'Deaths', 'Active', 'Discharged'])

total_cases = go.Bar(y=pd_df['Total_Cases'], name='Total Cases', hovertext=pd_df['State_UTs'])
deaths = go.Bar(y=pd_df['Deaths'], name='Deaths', hovertext=pd_df['State_UTs'])
active = go.Bar(y=pd_df['Active'], name='Active', hovertext=pd_df['State_UTs'])
discharged = go.Bar(y=pd_df['Discharged'], name='Discharged', hovertext=pd_df['State_UTs'])

# Add histrogram to figure
fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig.show()


In [63]:
# Notice Total Cases and Discharged graphs look relatively similar, this is because 
# Create group bar chart to have a side by side comparison between the Total Case values and Discharged values
fig = go.Figure([go.Bar( x=pd_df['State_UTs'], y=pd_df['Total_Cases'], name='Total Cases', hovertext=pd_df['State_UTs']),
                 go.Bar( x=pd_df['State_UTs'], y=pd_df['Discharged'], name='Discharged', hovertext=pd_df['State_UTs'])])

# Update figure so it is grouped and is given a title
fig.update_layout(barmode='group')
fig.update_layout(title='Total and Discharged Cases')
fig.show()

In [66]:
# Create piechart representing each charts 
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['Total Cases', 'Deaths', 'Active', 'Discharged'],
                    specs=[[{'type':'domain'}, {'type':'domain'}],
                           [{'type':'domain'}, {'type':'domain'}]])

total_cases = go.Pie(values=pd_df['Total_Cases'], name='Total Cases', labels=pd_df['State_UTs'])
deaths = go.Pie(values=pd_df['Deaths'], name='Deaths', labels=pd_df['State_UTs'])
active = go.Pie(values=pd_df['Active'], name='Active', labels=pd_df['State_UTs'])
discharged = go.Pie(values=pd_df['Discharged'], name='Discharged', labels=pd_df['State_UTs'])

fig.add_trace(total_cases, 1, 1)
fig.add_trace(deaths, 1, 2)
fig.add_trace(active, 2, 1)
fig.add_trace(discharged, 2, 2)

fig.update_traces(hoverinfo='percent+label')
fig.update_layout(showlegend=False)

fig.update_traces(textposition='inside')

fig = go.Figure(fig)
fig.show()

In [68]:
# Relationship between Total Cases and Deaths
fig = go.Figure([go.Scatter(x=pd_df['Total_Cases'], y=pd_df['Deaths'], mode='markers', text=pd_df['State_UTs'])])
fig.update_layout(title='Deaths vs Total Cases', xaxis_title='Total Cases', yaxis_title='Deaths')

In [71]:
# Create a numpy array to create a linear regression to display a simple stastical model
X = pd_df['Total_Cases'].to_numpy()
Y = pd_df['Deaths'].to_numpy()

# Create vector dimensions 
X.shape, Y.shape
((36,), (36,))

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X.reshape((len(X), 1)), Y)
(model.intercept_, model.coef_[0])

# Intercept value  and slope value

(-2382.512235937711, 0.013969146037293184)

In [72]:
# Calculate R squared value
model.score(X.reshape((len(X), 1)), Y)

0.8890729749873237

In [73]:
# Calculate fitted value for prediction for the linear regression based of the intercept value and slope value
predictions = model.predict(X.reshape((len(X), 1)))
predictions.shape

(36,)

In [74]:
# Plot the actual deaths and predicited deaths 
fig = go.Figure([go.Scatter(x=pd_df['Total_Cases'], y=pd_df['Deaths'], mode='markers', text=pd_df['State_UTs'], name='Actual Deaths'),
                 go.Scatter(x=pd_df['Total_Cases'], y=predictions, name='Predicted Deaths')])
fig.update_layout(title='Deaths vs Total Cases', xaxis_title='Total Cases', yaxis_title='Deaths')